# Evaluate static / moving state with Linear Discriminant Analysis model

In [1]:
import pickle
import pandas as pd
import numpy as np
from utils.helper import read_data
import utils.processing as processing
import matplotlib.pyplot as plt
FOLDER = "data/"
N_STATIC = 3
N_MOVING = 3

Open model

In [2]:
with open('models/pickle_model.pkl', 'rb') as file:
    clf_model = pickle.load(file)

### Load data

In [3]:
data_static = [read_data(FOLDER + "data_static_30_s_" + str(i) + ".csv") for i in range(N_STATIC)]
data_moving = [read_data(FOLDER + "data_moving_30_s_" + str(i) + ".csv") for i in range(N_MOVING)]


### Preprocess data

Static

In [4]:
phases_static = []
for i in range(N_STATIC):
    phases, abses, _, _ = processing.do_processing(data_static[i]) 
    phases_static.append(np.mean(phases, axis=0))

mean_phases_static = np.ravel(phases_static)
mean_phases_static = [abs(i) for i in mean_phases_static]

Moving

In [5]:
phases_moving = []
for i in range(N_MOVING):
    phases, abses, _, _ = processing.do_processing(data_moving[i]) 
    phases_moving.append(np.mean(phases, axis=0))

mean_phases_moving = np.ravel(phases_moving)
mean_phases_moving = [abs(i) for i in mean_phases_moving]

Reshape

In [6]:
mean_phases_static = np.array(mean_phases_static)
mean_phases_static = mean_phases_static.reshape(-1,1)
mean_phases_static_indexes = np.arange(len(mean_phases_static)).reshape(-1,1)
mean_phases_static_together = np.concatenate((mean_phases_static, mean_phases_static_indexes), axis=1)

mean_phases_moving = np.array(mean_phases_moving)
mean_phases_moving = mean_phases_moving.reshape(-1,1)
mean_phases_moving_indexes = np.arange(len(mean_phases_moving)).reshape(-1,1)
mean_phases_moving_together = np.concatenate((mean_phases_moving, mean_phases_moving_indexes), axis=1)

test_data = np.concatenate((mean_phases_static_together, mean_phases_moving_together))

Labels

In [7]:
indexes = np.arange(test_data.shape[0])
test_labels = indexes >= test_data.shape[0]/2
print(test_labels)

[False False False ...  True  True  True]


### Prediction

In [8]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

predicted_label = clf_model.predict(test_data)

print(classification_report(test_labels, predicted_label))

print('Accuracy : ' + str(accuracy_score(test_labels, predicted_label)))

              precision    recall  f1-score   support

       False       0.79      1.00      0.88     90000
        True       1.00      0.73      0.84     90000

    accuracy                           0.86    180000
   macro avg       0.89      0.86      0.86    180000
weighted avg       0.89      0.86      0.86    180000

Accuracy : 0.8630722222222222
